In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# 랜덤 시드 설정
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
class MCDropout3DCNN(tf.keras.Model):
    def __init__(self, input_shape, num_classes=5, dropout_rate=0.3):
        super(MCDropout3DCNN, self).__init__()
        
        # 첫 번째 3D 합성곱 블록
        self.conv1_1 = layers.Conv3D(32, kernel_size=3, activation='relu', padding='same')
        self.bn1_1 = layers.BatchNormalization()
        self.conv1_2 = layers.Conv3D(32, kernel_size=3, activation='relu', padding='same')
        self.bn1_2 = layers.BatchNormalization()
        self.pool1 = layers.MaxPool3D(pool_size=2)
        self.drop1 = layers.Dropout(dropout_rate)
        
        # 두 번째 3D 합성곱 블록
        self.conv2_1 = layers.Conv3D(64, kernel_size=3, activation='relu', padding='same')
        self.bn2_1 = layers.BatchNormalization()
        self.conv2_2 = layers.Conv3D(64, kernel_size=3, activation='relu', padding='same')
        self.bn2_2 = layers.BatchNormalization()
        self.pool2 = layers.MaxPool3D(pool_size=2)
        self.drop2 = layers.Dropout(dropout_rate)
        
        # 세 번째 3D 합성곱 블록
        self.conv3_1 = layers.Conv3D(128, kernel_size=3, activation='relu', padding='same',
                           kernel_regularizer=regularizers.l2(1e-4))
        self.bn3_1 = layers.BatchNormalization()
        self.conv3_2 = layers.Conv3D(128, kernel_size=3, activation='relu', padding='same',
                           kernel_regularizer=regularizers.l2(1e-4))
        self.bn3_2 = layers.BatchNormalization()
        self.pool3 = layers.MaxPool3D(pool_size=2)
        self.drop3 = layers.Dropout(dropout_rate)
        
        # 완전 연결 계층
        self.flatten = layers.Flatten()
        self.dense1 = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4))
        self.bn4 = layers.BatchNormalization()
        self.drop4 = layers.Dropout(dropout_rate)
        self.dense2 = layers.Dense(num_classes, activation='softmax')
    
    def call(self, inputs, training=False):
        # 첫 번째 블록
        x = self.conv1_1(inputs)
        x = self.bn1_1(x, training=training)
        x = self.conv1_2(x)
        x = self.bn1_2(x, training=training)
        x = self.pool1(x)
        x = self.drop1(x, training=training)
        
        # 두 번째 블록
        x = self.conv2_1(x)
        x = self.bn2_1(x, training=training)
        x = self.conv2_2(x)
        x = self.bn2_2(x, training=training)
        x = self.pool2(x)
        x = self.drop2(x, training=training)
        
        # 세 번째 블록
        x = self.conv3_1(x)
        x = self.bn3_1(x, training=training)
        x = self.conv3_2(x)
        x = self.bn3_2(x, training=training)
        x = self.pool3(x)
        x = self.drop3(x, training=training)
        
        # 완전 연결 계층
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.bn4(x, training=training)
        x = self.drop4(x, training=training)
        x = self.dense2(x)
        
        return x
    
    def build_graph(self):
        x = tf.keras.Input(shape=(16, 32, 32, 1))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))

In [ ]:
input_shape = (16, 32, 32, 1)  # (depth, height, width, channels)
mc_dropout_model = MCDropout3DCNN(input_shape)
mc_dropout_graph = mc_dropout_model.build_graph()
print("MC Dropout 모델 요약:")
mc_dropout_graph.summary()
    

In [ ]:
mc_dropout_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
def mc_dropout_predict(model, X, num_samples=30):
    predictions = []
    
    # 학습 모드로 설정하여 드롭아웃 활성화
    for _ in range(num_samples):
        y_pred = model(X, training=True)
        predictions.append(y_pred.numpy())
    
    # 예측값의 평균과 표준편차
    mean_prediction = np.mean(predictions, axis=0)
    std_prediction = np.std(predictions, axis=0)
    
    return mean_prediction, std_prediction

In [ ]:
def plot_uncertainty(mean_pred, std_pred, sample_idx=0):
    plt.figure(figsize=(12, 6))
    
    # 각 클래스별 예측 확률과 불확실성
    plt.subplot(1, 2, 1)
    num_classes = mean_pred.shape[1]
    classes = range(num_classes)
    
    plt.bar(classes, mean_pred[sample_idx], yerr=std_pred[sample_idx], capsize=10)
    plt.xlabel('Class')
    plt.ylabel('Probability')
    plt.title('Prediction Probability with Uncertainty')
    plt.xticks(classes)
    
    # 불확실성 분포
    plt.subplot(1, 2, 2)
    plt.hist(np.mean(std_pred, axis=1), bins=30)
    plt.axvline(std_pred[sample_idx].mean(), color='r', linestyle='--', 
                label=f'Sample #{sample_idx} uncertainty')
    plt.xlabel('Mean Uncertainty (Std)')
    plt.ylabel('Frequency')
    plt.title('Uncertainty Distribution')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_uncertainty_vs_accuracy(mean_pred, std_pred, y_true):
    # 예측 클래스와 실제 클래스
    pred_classes = np.argmax(mean_pred, axis=1)
    true_classes = np.argmax(y_true, axis=1)
    
    # 정확한 예측과 잘못된 예측 구분
    correct = pred_classes == true_classes
    
    # 각 샘플의 평균 불확실성
    mean_uncertainty = np.mean(std_pred, axis=1)
    
    plt.figure(figsize=(10, 6))
    plt.scatter(mean_uncertainty[correct], np.ones(np.sum(correct)), 
                label='Correct Predictions', alpha=0.5, color='blue')
    plt.scatter(mean_uncertainty[~correct], np.zeros(np.sum(~correct)), 
                label='Wrong Predictions', alpha=0.5, color='red')
    
    plt.xlabel('Mean Uncertainty (Std)')
    plt.ylabel('Prediction Correctness')
    plt.yticks([0, 1], ['Wrong', 'Correct'])
    plt.title('Relationship Between Uncertainty and Prediction Accuracy')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()


In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=50, batch_size=16):
    # 콜백 정의
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
    
    # 모델 학습
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    
    return history


In [ ]:
mc_dropout_model.save('mc_dropout_3dcnn_model')


In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 4))
    
    # 정확도 시각화
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # 손실 시각화
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
def evaluate_model_with_uncertainty(model, X_test, y_test):
    # MC Dropout을 사용한 베이지안 추론
    mean_pred, std_pred = mc_dropout_predict(model, X_test)
    pred_classes = np.argmax(mean_pred, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    
    # 정확도 계산
    accuracy = accuracy_score(true_classes, pred_classes)
    print(f"Test Accuracy: {accuracy:.4f}")
    
    # 혼동 행렬
    conf_matrix = confusion_matrix(true_classes, pred_classes)
    plt.figure(figsize=(8, 6))
    plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.tight_layout()
    plt.show()
    
    # 분류 보고서
    report = classification_report(true_classes, pred_classes)
    print("Classification Report:")
    print(report)
    
    # 불확실성 시각화
    plot_uncertainty(mean_pred, std_pred)
    
    # 불확실성과 정확도 관계 분석
    plot_uncertainty_vs_accuracy(mean_pred, std_pred, y_test)
    
    return mean_pred, std_pred


In [ ]:
loaded_model = tf.keras.models.load_model('mc_dropout_3dcnn_model')

